In [1]:
from operator import add
import os.path
import re
from pyspark import SparkContext

In [2]:
# set path and infile
home = os.path.expanduser('~')
inputPath = os.path.join('polatt', 'data', 'hr_2.txt')
infile = os.path.join(home, inputPath)

In [4]:
def wordCount(wordListRDD):
    return (wordListRDD
            .map(lambda x: (x, 1))
            .reduceByKey(add))

In [5]:
def removePunctuation(text):
    text = re.sub(r'[^\w\s]','',text)
    return text.lower().strip()

In [7]:
# read in words
wordsRDD = (sc
            .textFile(infile, 8)
            .map(removePunctuation)
            .cache())

# check that we have some text
print '\n'.join(wordsRDD
                .zipWithIndex()
                .map(lambda (l, num): '{0}: {1}'.format(num, l))
                .take(15))

0: congressional bills 114th congress
1: from the us government printing office
2: hr 2 introduced in house ih
3: 
4: 114th congress
5: 1st session
6: h r 2
7: 
8: to amend title xviii of the social security act to repeal the medicare
9: sustainable growth rate and strengthen medicare access by improving
10: physician payments and making other improvements to reauthorize the
11: childrens health insurance program and for other purposes
12: 
13: 
14: _______________________________________________________________________


In [8]:
# get all words and count
wordsRDD = (wordsRDD
            .flatMap(lambda x: x.split(' '))
            .filter(lambda x: len(x) >0))
print "\n\nWord count is: %i" % wordsRDD.count()



Word count is: 37964


In [10]:
# get top words
twords = 20
topWords = wordCount(wordsRDD).takeOrdered(twords,key=lambda x: -x[1])
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), topWords))

the: 2333
of: 1602
and: 1216
to: 911
in: 884
for: 835
a: 770
such: 654
under: 609
by: 505
shall: 412
or: 403
year: 344
secretary: 338
paragraph: 324
as: 308
is: 296
section: 273
with: 266
act: 261
